# Environment

In [113]:
import sys
import os
import lancedb
from langchain_community.vectorstores import LanceDB
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import AzureChatOpenAI
from langchain_openai import AzureOpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_core.output_parsers import StrOutputParser
from typing_extensions import TypedDict
from typing import List
from langgraph.graph import END, StateGraph
from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain.schema import Document
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from ragas import SingleTurnSample,evaluate,EvaluationDataset


from ragas.metrics import faithfulness


## Vectorstore

In [114]:
# from docx import Document

# def convert_bold_to_markdown(docx_file, output_file):
#     # 打開 Word 文件
#     doc = Document(docx_file)
#     markdown_text = ""
    
#     # 定義特定的 # 標題文字
#     main_headings = ["前言", "【春之卷】", "【夏之卷】", "【秋之卷】", "【冬之卷】"]

#     # 遍歷每段文字
#     for para in doc.paragraphs:
#         # 合併段落中的所有粗體文字，避免處理多餘的換行符號
#         bold_text = "".join([run.text.strip() for run in para.runs if run.bold])
        
#         # 如果有粗體文字
#         if bold_text:
#             if bold_text in main_headings:
#                 markdown_text += f"\n# {bold_text}\n"  # 特定文字為 #
#             else:
#                 markdown_text += f"\n## {bold_text}\n"  # 其餘文字為 ##
#         else:
#             # 如果整個段落不是粗體，直接加入段落內容
#             para_text = para.text.strip()
#             if para_text:  # 避免空段落
#                 markdown_text += para_text + "\n"

#     # 將轉換結果寫入文件
#     with open(output_file, "w", encoding="utf-8") as f:
#         f.write(markdown_text)

# # 測試
# convert_bold_to_markdown("東方本草-養出更好的人生.docx", "output.md")


In [115]:
# 讀取 Markdown 文件
def read_markdown(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    return content

# 測試
md_content = read_markdown("output.md")

In [116]:

# 定義分割的標題層級
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

# 初始化 Markdown 分割器
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

# 分割 Markdown 文本
md_header_splits = markdown_splitter.split_text(md_content)

# 將分割結果存儲到 processed_data
processed_data = []
for chunk in md_header_splits:
    metadata = chunk.metadata
    content = chunk.page_content

    # 如果 Header 2 存在，將其加到內文最前面
    if metadata.get("Header 2"):
        content = f"{metadata['Header 2']}，{content}"
    
    processed_data.append({
        'content': content,
        'metadata': metadata
    })

In [117]:
sys.path.append("..")
from pke_zh import TfIdf


# 更新 processed_data 並加入關鍵字
for item in processed_data:
    content = item['content']
    metadata = item['metadata']

    # 提取關鍵字並更新 Metadata
    keywords = TfIdf_m.extract(item['content'])

    # 只保留前三個關鍵字，不包含分數
    keywords_only = [keyword[0] for keyword in keywords[:3]]

    metadata['Keywords'] = keywords_only
    item['metadata'] = metadata

# 將整理好的數據構造為 documents 格式
documents = [
    {
        "text": item["content"],  # 使用更新後的內容
        "metadata": item["metadata"]  # 包含已更新的 Metadata
    }
    for item in processed_data
]

# 提取並輸出結果
for doc in documents:
    print(f"Text: {doc['text']}")
    print(f"Metadata: {doc['metadata']}")


Text: 02【春之卷】04【夏之卷】12【秋之卷】20【冬之卷】28
養生輕鬆做養精神調飲食 訓練身體 掃描QR code看郭大維醫師教你如何四季養生
Metadata: {'Header 1': '前言', 'Keywords': ['養生', '02', '04']}
Text: 春夏養陽 秋冬養陰，「藥補不如食補，食補不如睡補」，這是大家耳熟能詳的一句話，「睡補」符合古人所說的「日出而作，日落而息」習慣，也能達到「春生、夏長、秋收、冬藏」的概念。郭大維中醫師進一步解釋，春生、夏長、秋收、冬藏是四季發展規律，意思是萬物春天萌生，夏天滋長，秋天收穫，冬天儲藏。春夏陽氣散發於外，因此春夏宜養陽，而秋、冬時，就要跟著萬物一起收斂自己，把陰液（所有能濡潤身體的物質）留在體內，以準備過冬。
Metadata: {'Header 1': '前言', 'Header 2': '春夏養陽 秋冬養陰', 'Keywords': ['春夏', '食補', '睡補']}
Text: 當心大自然六氣成致病六邪，「風、寒、暑、濕、燥、火」是原本就存在於大自然的「六氣」，不過，當六氣過盛時，就會變成致病的「六邪」。春暖、夏熱、秋涼、冬寒，一年四季周而復始。氣候的演變和環境中的六氣互有關聯，每個季節養生的重點也會因氣候而有所差異，例如春季時最易遭受風邪侵犯，夏季則應提防暑邪與濕邪的侵擾。中醫說：「正氣存內，邪不可干，邪之所湊，其氣必虛」，意思是當身體免疫力好時，六邪就不易入侵體內，相反的，若身體太過虛弱，即使是自然界的變化，也可能成為致病因子。因時養生，按照季節與陰陽變化的規律，從事相對應的養生方法，不但能達到「天、地、人合一」的境界，也能讓健康養生更事半功倍！運用老祖宗的智慧，將東方本草的知識，以及古代東方養生的概念融入於生活之中，不但可以預防疾病發生，也能讓身心靈更自在、更健康。養生，其實也可以很簡單，期望大家都能養出更好的人生！
Metadata: {'Header 1': '前言', 'Header 2': '當心大自然六氣成致病六邪', 'Keywords': ['養生', '致病', '氣候']}
Text: 你是否有以下的季節性小症狀？，春天呼吸道不適 夏天濕疹 秋天乾咳或皮膚癢 冬天慢性病況加重 季節過敏 換季鼻塞。季節會影響身體的變化，許多不起眼的症狀都是因應環境、氣候

In [118]:
documents = [
    Document(page_content=item['content'], metadata=item['metadata'])
    for item in processed_data
]

In [119]:
bm25_retriever = BM25Retriever.from_documents(documents)
bm25_retriever.k = 2


In [120]:
# 定義要使用哪個Embedding model 將chunk內的文字轉為向量
embeddings = AzureOpenAIEmbeddings(
    azure_deployment="text-embedding-3-small", # 另外一個embedding模型
    openai_api_version="2023-05-15", # 在URL裡面
)


In [121]:
data = [
    {
        "vector": embeddings.embed_query(doc.page_content),  # 使用 page_content 屬性
        "text": doc.page_content,  # 文本內容
        "metadata": doc.metadata,  # 元數據
        "id": str(index),  # 確保唯一性
    }
    for index, doc in enumerate(documents)  # documents 應為 Document 類型列表
]

db = lancedb.connect("/tmp/lancedb")
table = db.create_table("pandas_docs", data=data, mode="overwrite")


In [122]:
# 查询表中的所有数据
all_data = table.to_pandas()  # 转换为 Pandas DataFrame，方便查看和分析

for index, row in all_data.iterrows():
    print(f"ID: {row['id']}")
    print(f"Text: {row['text']}")
    print(f"Metadata: {row['metadata']}")
    print(f"Vector: {row['vector'][:5]}...")  # 只显示向量的前5个值
    print("-" * 50)


ID: 0
Text: 02【春之卷】04【夏之卷】12【秋之卷】20【冬之卷】28
養生輕鬆做養精神調飲食 訓練身體 掃描QR code看郭大維醫師教你如何四季養生
Metadata: {'Header 1': '前言', 'Header 2': None, 'Keywords': array(['養生', '02', '04'], dtype=object)}
Vector: [0.04732722 0.04651244 0.03666521 0.0271439  0.02121926]...
--------------------------------------------------
ID: 1
Text: 春夏養陽 秋冬養陰，「藥補不如食補，食補不如睡補」，這是大家耳熟能詳的一句話，「睡補」符合古人所說的「日出而作，日落而息」習慣，也能達到「春生、夏長、秋收、冬藏」的概念。郭大維中醫師進一步解釋，春生、夏長、秋收、冬藏是四季發展規律，意思是萬物春天萌生，夏天滋長，秋天收穫，冬天儲藏。春夏陽氣散發於外，因此春夏宜養陽，而秋、冬時，就要跟著萬物一起收斂自己，把陰液（所有能濡潤身體的物質）留在體內，以準備過冬。
Metadata: {'Header 1': '前言', 'Header 2': '春夏養陽 秋冬養陰', 'Keywords': array(['春夏', '食補', '睡補'], dtype=object)}
Vector: [-0.00519569  0.01817207 -0.00474858  0.05069267  0.00927618]...
--------------------------------------------------
ID: 2
Text: 當心大自然六氣成致病六邪，「風、寒、暑、濕、燥、火」是原本就存在於大自然的「六氣」，不過，當六氣過盛時，就會變成致病的「六邪」。春暖、夏熱、秋涼、冬寒，一年四季周而復始。氣候的演變和環境中的六氣互有關聯，每個季節養生的重點也會因氣候而有所差異，例如春季時最易遭受風邪侵犯，夏季則應提防暑邪與濕邪的侵擾。中醫說：「正氣存內，邪不可干，邪之所湊，其氣必虛」，意思是當身體免疫力好時，六邪就不易入侵體內，相反的，若身體太過虛弱，即使是自然界的變化，也可能成為致病因子。因時養生，按照

In [123]:
# 初始化 LanceDB 檢索器
docsearch = LanceDB.from_documents(documents, embeddings, connection=db)
retriever_lancedb = docsearch.as_retriever(search_kwargs={"k": 2})

# 初始化混合檢索器
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, retriever_lancedb], weights=[0.4, 0.6]
)

## Web Search Tool

In [124]:
web_search_tool = TavilySearchResults()

## LLMs

### Question Router

In [125]:
# 定義兩個工具的 DataModel
class web_search(BaseModel):
    """
    網路搜尋工具。若問題與養身"無關"，請使用web_search工具搜尋解答。
    """
    query: str = Field(description="使用網路搜尋時輸入的問題")

class vectorstore(BaseModel):
    """
    紀錄關於養身的向量資料庫工具。若問題與養身有關，請使用此工具搜尋解答。
    """
    query: str = Field(description="搜尋向量資料庫時輸入的問題")


# Prompt Template
instruction = """
你是將使用者問題導向向量資料庫或網路搜尋的專家。
向量資料庫包含有關養身文件。對於這些主題的問題，請使用向量資料庫工具。其他情況則使用網路搜尋工具。
"""
route_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",instruction),
        ("human", "{question}"),
    ]
)

# Route LLM with tools use
llm = AzureChatOpenAI(
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
    temperature=0
)

structured_llm_router = llm.bind_tools(tools=[web_search, vectorstore])

# 使用 LCEL 語法建立 chain
question_router = route_prompt | structured_llm_router

In [126]:
# 測試 Route 功能
response = question_router.invoke({"question": "東京的經緯度是多少?"})
print(response.additional_kwargs['tool_calls'])
response = question_router.invoke({"question": "哪個季節適合養肺?"}) # 問題要明確
print(response.additional_kwargs['tool_calls'])
response = question_router.invoke({"question": "你好"}) # llm直接回答
print('tool_calls' in response.additional_kwargs)

[{'id': 'call_VKbkIgU9B0LAKL71sOSChl2z', 'function': {'arguments': '{"query":"東京的經緯度"}', 'name': 'web_search'}, 'type': 'function'}]
[{'id': 'call_QCKlAr8krQQxrZ8P4NCHtBZV', 'function': {'arguments': '{"query":"哪個季節適合養肺"}', 'name': 'vectorstore'}, 'type': 'function'}]
False


### RAG Responder

In [127]:
# Prompt Template
instruction = """
你是一位負責處理使用者問題的助手，請利用提取出來的文件內容來回應問題。
若問題的答案無法從文件內取得，請直接回覆你不知道，禁止虛構答案。
注意：請確保答案的準確性。
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system",instruction),
        ("system","文件: \n\n {documents}"),
        ("human","問題: {question}"),
    ]
)

# LLM & chain
rag_chain = prompt | llm | StrOutputParser()

In [128]:
# 測試 rag_chain 功能
question = "五臟?"
docs = ensemble_retriever.invoke(question)
generation = rag_chain.invoke({"documents": docs, "question": question})
print(generation)

五臟是指中醫理論中的肝、心、脾、肺、腎，這五個臟腑對應於五行的木、火、土、金、水。


### Plain LLM

In [129]:
# Prompt Teamplate
instruction = """
你是一位負責處理使用者問題的助手，請利用你的知識來回應問題。
回應問題時請確保答案的準確性，勿虛構答案。
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system",instruction),
        ("human","問題: {question}"),
    ]
)

# LLM & chain
llm_chain = prompt | llm | StrOutputParser()

In [130]:
# 測試 llm_chain 功能
question = "Hi how are you?"
generation = llm_chain.invoke({"question": question})
print(generation)

Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?


### Retrieval Grader

In [131]:
class GradeDocuments(BaseModel):
    """
    確認提取文章與問題是否有關(yes/no)
    """

    binary_score: str = Field(description="請問文章與問題是否相關。('yes' or 'no')")

# Prompt Template
instruction = """
你是一個評分的人員，負責評估文件與使用者問題的關聯性。
如果文件包含與使用者問題相關的關鍵字或語意，則將其評為相關。
輸出 'yes' or 'no' 代表文件與問題的相關與否。
"""
grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",instruction),
        ("human", "文件: \n\n {document} \n\n 使用者問題: {question}"),
    ]
)

# Grader LLM
structured_llm_grader = llm.with_structured_output(GradeDocuments)

# 使用 LCEL 語法建立 chain
retrieval_grader = grade_prompt | structured_llm_grader

In [132]:
# 測試 grader 功能
question = "秋季可以吃那些潤肺的食物?"
docs = ensemble_retriever.invoke(question)
doc_txt = docs[0].page_content
response =  retrieval_grader.invoke({"question": question, "document": doc_txt})
print(response)

binary_score='yes'


In [133]:
docs

[Document(metadata={'page': None, 'source': None}, page_content='秋季吃什麼最潤肺，梨子、百合、羅漢果、人參、茯苓秋季潤肺養呼吸道秋季吃什麼最潤肺'),
 Document(metadata={'page': None, 'source': None}, page_content='秋天肺與呼吸道，易乾燥，可吃隔水蒸煮的梨子潤肺，並多呼吸新鮮的空氣保養口鼻。梨子、百合、羅漢果、人參、茯苓等，都是可補養肺氣與呼吸道、潤燥的食材。'),
 Document(metadata={'Header 1': '【冬之卷】', 'Header 2': '發行單位', 'Keywords': ['疾病', '發行', '單位']}, page_content='發行單位，Nutrilite紐崔萊執行製作｜良醫健康網※本內容僅提供相關疾病之醫學及衛教資訊；關於個人健康狀況或疾病治療方面的問題，建議向醫護人員諮詢專業意見。'),
 Document(metadata={'Header 1': '【夏之卷】', 'Header 2': '初夏養心長夏養脾胃', 'Keywords': ['脾胃', '中醫', '臟腑']}, page_content='初夏養心長夏養脾胃，相較於西醫的心臟，中醫的「心」涵蓋的範圍更廣泛。「心主血脈」，表示全身的血脈，也就是所有的血管都歸心所管。不過，《黃帝內經》也指出：「心者君主之官也，神明出焉」，意思是心也主宰著人的精神活動與意識思維，由此可知，古人認為心和大腦活動也有關係。郭大維中醫師強調，趁夏季時將心這個臟腑調理好，受惠不只是心臟及心血管，包括大腦及思緒也變得更清晰、有精神。夏天的末端濕氣會變得特別重，因此中醫將夏末到初秋這段時間從夏季裡劃分出來，稱為「長夏」。雖然夏季養生的重點在養心，但長夏時濕氣較重，所以還必須特別注意脾胃的健康。\n長夏養生首重健脾利濕，因為體內又濕又熱，特別容易導致臟腑上火，出現心火過旺的現象，也會影響心血管循環。因此，長夏時如果能將脾胃調理好，讓體內代謝水分的功能正常，就能清熱降火，進而達到保養心血管的效用。中醫認為「脾為後天之本，為氣血生化之源」，意思是脾胃在消化食物之後，會將這些營養物質運送、分布至各個臟腑，讓它們得到滋養。反之，若脾胃功能失調，就無法將食

### Answer Grader

In [134]:
class GradeAnswer(BaseModel):
    """
    確認答案是否可回應問題
    """

    binary_score: str = Field(description="答案是否回應問題。('yes' or 'no')")

# Prompt Template
instruction = """
你是一個評分的人員，負責確認答案是否回應了問題。
輸出 'yes' or 'no'。 'Yes' 代表答案確實回應了問題， 'No' 則代表答案並未回應問題。
"""
# Prompt
answer_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",instruction),
        ("human", "使用者問題: \n\n {question} \n\n 答案: {generation}"),
    ]
)

# LLM with function call
structured_llm_grader = llm.with_structured_output(GradeAnswer)

# 使用 LCEL 語法建立 chain
answer_grader = answer_prompt | structured_llm_grader

In [135]:
#測試 grader 功能
question = "甚麼季節最適合養肺?"
docs = ensemble_retriever.invoke(question)
generation = rag_chain.invoke({"documents": docs, "question": question})
answer_grader.invoke({"question": question,"generation": generation})

GradeAnswer(binary_score='yes')

# Graph

## Graph state

In [136]:

class GraphState(TypedDict):
    """
    State of graph.

    Attributes:
        question: question
        generation: LLM generation
        documents: list of documents
    """
    question : str
    generation : str
    documents : List[str]

## Nodes / Conditional edges

In [137]:
def retrieve(state):
    """
    Retrieve documents related to the question.

    Args:
        state (dict):  The current state graph

    Returns:
        state (dict): New key added to state, documents, that contains list of related documents.
    """

    print("---RETRIEVE---")
    question = state["question"]

    # Retrieval
    documents = ensemble_retriever.invoke(question)
    
    print(documents)
    return {"documents":documents, "question":question}

def web_search(state):
    """
    Web search based on the re-phrased question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with appended web results
    """

    print("---WEB SEARCH---")
    question = state["question"]
    
    documents = state["documents"] if state["documents"] else []
    # print(documents)
    # state.get(retriever)
    # Web search
    docs = web_search_tool.invoke({"query": question})
    web_results = [Document(page_content=d["content"]) for d in docs]
    
    # print(web_results)
    # print("web_search",web_results)

    # web_results含有documents
    documents = documents + web_results
    print(documents)
    return {"documents": documents, "question": question}

def retrieval_grade(state):
    """
    filter retrieved documents based on question.

    Args:
        state (dict):  The current state graph

    Returns:
        state (dict): New key added to state, documents, that contains list of related documents.
    """

    # Grade documents
    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")

    documents = state["documents"]
    question = state["question"]

    # Score each doc
    filtered_docs = []
    for d in documents:
        score = retrieval_grader.invoke({"question": question, "document": d.page_content})
        grade = score.binary_score
        if grade == "yes":
            print("  -GRADE: DOCUMENT RELEVANT-")
            filtered_docs.append(d)
        else:
            print("  -GRADE: DOCUMENT NOT RELEVANT-")
            continue
    return {"documents": filtered_docs, "question": question}

def rag_generate(state):
    """
    Generate answer using  vectorstore / web search

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """

    print("---GENERATE IN RAG MODE---")
    question = state["question"]
    documents = state["documents"]

    # RAG generation
    generation = rag_chain.invoke({"documents": documents, "question": question})
    return {"documents": documents, "question": question, "generation": generation}

def plain_answer(state):
    """
    Generate answer using the LLM without vectorstore.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """

    print("---GENERATE PLAIN ANSWER---")
    question = state["question"]
    generation = llm_chain.invoke({"question": question})
    return {"question": question, "generation": generation}


### Edges ###
def route_question(state):
    """
    Route question to web search or RAG.

    Args:
        state (dict): The current graph state

    Returns:
        str: Next node to call
    """

    print("---ROUTE QUESTION---")
    question = state["question"]
    source = question_router.invoke({"question": question})

    # Fallback to plain LLM or raise error if no decision
    if "tool_calls" not in source.additional_kwargs:
        print("  -ROUTE TO PLAIN LLM-")
        return "plain_answer"
    if len(source.additional_kwargs["tool_calls"]) == 0:
      raise "Router could not decide source"

    # Choose datasource
    datasource = source.additional_kwargs["tool_calls"][0]["function"]["name"]
    if datasource == 'web_search':
        print("  -ROUTE TO WEB SEARCH-")
        return "web_search"
    elif datasource == 'vectorstore':
        print("  -ROUTETO VECTORSTORE-")
        return "vectorstore"

def route_retrieval(state):
    """
    Determines whether to generate an answer, or use websearch.

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    print("---ROUTE RETRIEVAL---")
    filtered_documents = state["documents"]

    if not filtered_documents:
        # All documents have been filtered check_relevance
        print("  -DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, ROUTE TO WEB SEARCH-")
        return "web_search"
    else:
        # We have relevant documents, so generate answer
        print("  -DECISION: GENERATE WITH RAG LLM-")
        return "rag_generate"


In [138]:
def grade_rag_generation(state):
    """
    Determines whether the generation is grounded in the document and answers question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Decision for next node to call
    """

    print("---CHECK HALLUCINATIONS---")
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]
    
    print("State content:", type(state),state)
    contexts = [doc.page_content for doc in documents]  # Extract page content from documents
    sample = SingleTurnSample(
        user_input=question,
        response=generation,
        retrieved_contexts=contexts,
    )
    dataset = EvaluationDataset([sample])
    print("State content:", type(dataset))
    print(dataset)
    faithfulness_score = evaluate(dataset=dataset,metrics=[faithfulness],llm=llm)
    print(type(faithfulness_score),faithfulness_score)
    score = float(faithfulness_score['faithfulness'][0])
    print(type(score),score)
    if score < 0.5:
        print("  -DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY-")
        return "not supported"

    else:
        print("  -DECISION: GENERATION IS GROUNDED IN DOCUMENTS-")
        # Check question-answering
        print("---GRADE GENERATION vs QUESTION---")
        score = answer_grader.invoke({"question": question,"generation": generation})
        grade = score.binary_score
        if grade == "yes":
            print("  -DECISION: GENERATION ADDRESSES QUESTION-")
            return "useful"
        else:
            print("  -DECISION: GENERATION DOES NOT ADDRESS QUESTION-")
            return "not useful"

## Build Graph

In [139]:
workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("web_search", web_search) # web search
workflow.add_node("retrieve", retrieve) # retrieve
workflow.add_node("retrieval_grade", retrieval_grade) # retrieval grade
workflow.add_node("rag_generate", rag_generate) # rag
workflow.add_node("plain_answer", plain_answer) # llm

# Build graph
workflow.set_conditional_entry_point(
    route_question,
    {
        "web_search": "web_search",
        "vectorstore": "retrieve",
        "plain_answer": "plain_answer",
    },
)
workflow.add_edge("retrieve", "retrieval_grade")
workflow.add_edge("web_search", "retrieval_grade")
workflow.add_conditional_edges(
    "retrieval_grade",
    route_retrieval,
    {
        "web_search": "web_search",
        "rag_generate": "rag_generate",
    },
)
workflow.add_conditional_edges(
    "rag_generate",
    grade_rag_generation,
    {
        "not supported": "rag_generate", # Hallucinations: re-generate
        "not useful": "web_search", # Fails to answer question: fall-back to web-search
        "useful": END,
    },
)
workflow.add_edge("plain_answer", END)

# Compile
app = workflow.compile()

### 實際測試

In [140]:
def run(question):
    inputs = {"question": question,"documents":None}
    for output in app.stream(inputs):
        print("\n")

    # Final generation
    if 'rag_generate' in output.keys():
        print(output['rag_generate']['generation'])
    elif 'plain_answer' in output.keys():
        print(output['plain_answer']['generation'])

In [141]:
run("請選擇正確的秋季養肺方式(可複選) 1. 洗高溫 熱水澡防感冒 2. 保養口鼻 呼吸好空氣 3. 吃梨潤肺 需隔水蒸煮 4. 起床喝冰水 滋潤肺部")

---ROUTE QUESTION---
  -ROUTETO VECTORSTORE-
---RETRIEVE---
[Document(metadata={'page': None, 'source': None}, page_content='跟著中醫師學養肺肺，最討厭燥氣，因為太過乾燥的氣候會傷及肺陰。想要養肺，一定要把握滋陰潤燥的原則，建議秋季可以多吃潤肺的食物，例如銀耳、山藥等。除了較滋潤的食物之外，還要記得多補充水分，乾淨的水或茶飲都是不錯的選擇。郭大維中醫師提到，蜂蜜水不但可以潤燥、清熱、止咳，又具有潤腸的效用，可以改善秋季乾咳、喉嚨卡卡、皮膚乾燥及便秘等諸多問題。肺主一身之氣，保持乾燥、好品質的空氣，對於肺部的健康非常重要，尤其臺灣屬於潮濕的海島型氣候，除濕工作更不能馬虎。想要維告別悲秋 肺更平靜持潔淨、乾爽的空氣，空氣清淨機及除濕機一定不能少，將室內濕度維持在40%至60%之間，呼吸道就會感覺比較舒適。此外，在早晚空氣品質較佳的時候，打開室內的窗戶讓空氣對流一下，也可以讓空氣更清新。\n工作忙碌過度勞累時，可以走出大自然，讓自己適度放鬆、疏通肺氣。'), Document(metadata={'page': None, 'source': None}, page_content='秋季吃什麼最潤肺，梨子、百合、羅漢果、人參、茯苓秋季潤肺養呼吸道秋季吃什麼最潤肺'), Document(metadata={'Header 1': '【冬之卷】', 'Header 2': '發行單位', 'Keywords': ['疾病', '發行', '單位']}, page_content='發行單位，Nutrilite紐崔萊執行製作｜良醫健康網※本內容僅提供相關疾病之醫學及衛教資訊；關於個人健康狀況或疾病治療方面的問題，建議向醫護人員諮詢專業意見。'), Document(metadata={'Header 1': '【夏之卷】', 'Header 2': '初夏養心長夏養脾胃', 'Keywords': ['脾胃', '中醫', '臟腑']}, page_content='初夏養心長夏養脾胃，相較於西醫的心臟，中醫的「心」涵蓋的範圍更廣泛。「心主血脈」，表示全身的血脈，也就是所有的血管都歸心所管。不過，《黃帝內經》也指出：「心者君主之官也，神明出焉」，意思是

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

<class 'ragas.dataset_schema.EvaluationResult'> {'faithfulness': 0.0000}
<class 'float'> 0.0
  -DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY-


---GENERATE IN RAG MODE---
---CHECK HALLUCINATIONS---
State content: <class 'dict'> {'documents': [Document(metadata={'page': None, 'source': None}, page_content='跟著中醫師學養肺肺，最討厭燥氣，因為太過乾燥的氣候會傷及肺陰。想要養肺，一定要把握滋陰潤燥的原則，建議秋季可以多吃潤肺的食物，例如銀耳、山藥等。除了較滋潤的食物之外，還要記得多補充水分，乾淨的水或茶飲都是不錯的選擇。郭大維中醫師提到，蜂蜜水不但可以潤燥、清熱、止咳，又具有潤腸的效用，可以改善秋季乾咳、喉嚨卡卡、皮膚乾燥及便秘等諸多問題。肺主一身之氣，保持乾燥、好品質的空氣，對於肺部的健康非常重要，尤其臺灣屬於潮濕的海島型氣候，除濕工作更不能馬虎。想要維告別悲秋 肺更平靜持潔淨、乾爽的空氣，空氣清淨機及除濕機一定不能少，將室內濕度維持在40%至60%之間，呼吸道就會感覺比較舒適。此外，在早晚空氣品質較佳的時候，打開室內的窗戶讓空氣對流一下，也可以讓空氣更清新。\n工作忙碌過度勞累時，可以走出大自然，讓自己適度放鬆、疏通肺氣。'), Document(metadata={'page': None, 'source': None}, page_content='秋季吃什麼最潤肺，梨子、百合、羅漢果、人參、茯苓秋季潤肺養呼吸道秋季吃什麼最潤肺')], 'question': '請選擇正確的秋季養肺方式(可複選) 1. 洗高溫 熱水澡防感冒 2. 保養口鼻 呼吸好空氣 3. 吃梨潤肺 需隔水蒸煮 4. 起床喝冰水 滋潤肺部', 'generation': '正確的秋季養肺方式是：\n\n2. 保養口鼻 呼吸好空氣  \n3. 吃梨潤肺 需隔水蒸煮  \n\n選項1和選項4不符合養肺的原則。'}
State content: <class 'raga

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

<class 'ragas.dataset_schema.EvaluationResult'> {'faithfulness': 0.0000}
<class 'float'> 0.0
  -DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY-


---GENERATE IN RAG MODE---
---CHECK HALLUCINATIONS---
State content: <class 'dict'> {'documents': [Document(metadata={'page': None, 'source': None}, page_content='跟著中醫師學養肺肺，最討厭燥氣，因為太過乾燥的氣候會傷及肺陰。想要養肺，一定要把握滋陰潤燥的原則，建議秋季可以多吃潤肺的食物，例如銀耳、山藥等。除了較滋潤的食物之外，還要記得多補充水分，乾淨的水或茶飲都是不錯的選擇。郭大維中醫師提到，蜂蜜水不但可以潤燥、清熱、止咳，又具有潤腸的效用，可以改善秋季乾咳、喉嚨卡卡、皮膚乾燥及便秘等諸多問題。肺主一身之氣，保持乾燥、好品質的空氣，對於肺部的健康非常重要，尤其臺灣屬於潮濕的海島型氣候，除濕工作更不能馬虎。想要維告別悲秋 肺更平靜持潔淨、乾爽的空氣，空氣清淨機及除濕機一定不能少，將室內濕度維持在40%至60%之間，呼吸道就會感覺比較舒適。此外，在早晚空氣品質較佳的時候，打開室內的窗戶讓空氣對流一下，也可以讓空氣更清新。\n工作忙碌過度勞累時，可以走出大自然，讓自己適度放鬆、疏通肺氣。'), Document(metadata={'page': None, 'source': None}, page_content='秋季吃什麼最潤肺，梨子、百合、羅漢果、人參、茯苓秋季潤肺養呼吸道秋季吃什麼最潤肺')], 'question': '請選擇正確的秋季養肺方式(可複選) 1. 洗高溫 熱水澡防感冒 2. 保養口鼻 呼吸好空氣 3. 吃梨潤肺 需隔水蒸煮 4. 起床喝冰水 滋潤肺部', 'generation': '正確的秋季養肺方式是：\n\n2. 保養口鼻 呼吸好空氣  \n3. 吃梨潤肺 需隔水蒸煮  \n\n選項1和選項4不符合養肺的原則。'}
State content: <class 'raga

...

<class 'ragas.dataset_schema.EvaluationResult'> {'faithfulness': 0.0000}
<class 'float'> 0.0
  -DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY-


---GENERATE IN RAG MODE---
---CHECK HALLUCINATIONS---
State content: <class 'dict'> {'documents': [Document(metadata={'page': None, 'source': None}, page_content='跟著中醫師學養肺肺，最討厭燥氣，因為太過乾燥的氣候會傷及肺陰。想要養肺，一定要把握滋陰潤燥的原則，建議秋季可以多吃潤肺的食物，例如銀耳、山藥等。除了較滋潤的食物之外，還要記得多補充水分，乾淨的水或茶飲都是不錯的選擇。郭大維中醫師提到，蜂蜜水不但可以潤燥、清熱、止咳，又具有潤腸的效用，可以改善秋季乾咳、喉嚨卡卡、皮膚乾燥及便秘等諸多問題。肺主一身之氣，保持乾燥、好品質的空氣，對於肺部的健康非常重要，尤其臺灣屬於潮濕的海島型氣候，除濕工作更不能馬虎。想要維告別悲秋 肺更平靜持潔淨、乾爽的空氣，空氣清淨機及除濕機一定不能少，將室內濕度維持在40%至60%之間，呼吸道就會感覺比較舒適。此外，在早晚空氣品質較佳的時候，打開室內的窗戶讓空氣對流一下，也可以讓空氣更清新。\n工作忙碌過度勞累時，可以走出大自然，讓自己適度放鬆、疏通肺氣。'), Document(metadata={'page': None, 'source': None}, page_content='秋季吃什麼最潤肺，梨子、百合、羅漢果、人參、茯苓秋季潤肺養呼吸道秋季吃什麼最潤肺')], 'question': '請選擇正確的秋季養肺方式(可複選) 1. 洗高溫 熱水澡防感冒 2. 保養口鼻 呼吸好空氣 3. 吃梨潤肺 需隔水蒸煮 4. 起床喝冰水 滋潤肺部', 'generation': '正確的秋季養肺方式是：\n\n2. 保養口鼻 呼吸好空氣  \n3. 吃梨潤肺 需隔水蒸煮  \n\n選項1和4不符合養肺的原則。'}
State content: <class 'ragas.

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

<class 'ragas.dataset_schema.EvaluationResult'> {'faithfulness': 0.0000}
<class 'float'> 0.0
  -DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY-


---GENERATE IN RAG MODE---
---CHECK HALLUCINATIONS---
State content: <class 'dict'> {'documents': [Document(metadata={'page': None, 'source': None}, page_content='跟著中醫師學養肺肺，最討厭燥氣，因為太過乾燥的氣候會傷及肺陰。想要養肺，一定要把握滋陰潤燥的原則，建議秋季可以多吃潤肺的食物，例如銀耳、山藥等。除了較滋潤的食物之外，還要記得多補充水分，乾淨的水或茶飲都是不錯的選擇。郭大維中醫師提到，蜂蜜水不但可以潤燥、清熱、止咳，又具有潤腸的效用，可以改善秋季乾咳、喉嚨卡卡、皮膚乾燥及便秘等諸多問題。肺主一身之氣，保持乾燥、好品質的空氣，對於肺部的健康非常重要，尤其臺灣屬於潮濕的海島型氣候，除濕工作更不能馬虎。想要維告別悲秋 肺更平靜持潔淨、乾爽的空氣，空氣清淨機及除濕機一定不能少，將室內濕度維持在40%至60%之間，呼吸道就會感覺比較舒適。此外，在早晚空氣品質較佳的時候，打開室內的窗戶讓空氣對流一下，也可以讓空氣更清新。\n工作忙碌過度勞累時，可以走出大自然，讓自己適度放鬆、疏通肺氣。'), Document(metadata={'page': None, 'source': None}, page_content='秋季吃什麼最潤肺，梨子、百合、羅漢果、人參、茯苓秋季潤肺養呼吸道秋季吃什麼最潤肺')], 'question': '請選擇正確的秋季養肺方式(可複選) 1. 洗高溫 熱水澡防感冒 2. 保養口鼻 呼吸好空氣 3. 吃梨潤肺 需隔水蒸煮 4. 起床喝冰水 滋潤肺部', 'generation': '根據文件內容，正確的秋季養肺方式是：\n\n2. 保養口鼻 呼吸好空氣\n3. 吃梨潤肺 需隔水蒸煮\n\n選項1和4並未在文件中提到，因此無法確認其正確性。'}
State content: 

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

<class 'ragas.dataset_schema.EvaluationResult'> {'faithfulness': 0.5000}
<class 'float'> 0.5
  -DECISION: GENERATION IS GROUNDED IN DOCUMENTS-
---GRADE GENERATION vs QUESTION---
  -DECISION: GENERATION ADDRESSES QUESTION-


根據文件內容，正確的秋季養肺方式是：

2. 保養口鼻 呼吸好空氣
3. 吃梨潤肺 需隔水蒸煮

選項1和4並未在文件中提到，因此無法確認其正確性。


In [142]:
run("秋天吃甚麼潤肺?")

---ROUTE QUESTION---
  -ROUTETO VECTORSTORE-
---RETRIEVE---
[Document(metadata={'page': None, 'source': None}, page_content='秋季吃什麼最潤肺，梨子、百合、羅漢果、人參、茯苓秋季潤肺養呼吸道秋季吃什麼最潤肺'), Document(metadata={'page': None, 'source': None}, page_content='秋天肺與呼吸道，易乾燥，可吃隔水蒸煮的梨子潤肺，並多呼吸新鮮的空氣保養口鼻。梨子、百合、羅漢果、人參、茯苓等，都是可補養肺氣與呼吸道、潤燥的食材。'), Document(metadata={'Header 1': '【冬之卷】', 'Header 2': '發行單位', 'Keywords': ['疾病', '發行', '單位']}, page_content='發行單位，Nutrilite紐崔萊執行製作｜良醫健康網※本內容僅提供相關疾病之醫學及衛教資訊；關於個人健康狀況或疾病治療方面的問題，建議向醫護人員諮詢專業意見。'), Document(metadata={'Header 1': '【夏之卷】', 'Header 2': '初夏養心長夏養脾胃', 'Keywords': ['脾胃', '中醫', '臟腑']}, page_content='初夏養心長夏養脾胃，相較於西醫的心臟，中醫的「心」涵蓋的範圍更廣泛。「心主血脈」，表示全身的血脈，也就是所有的血管都歸心所管。不過，《黃帝內經》也指出：「心者君主之官也，神明出焉」，意思是心也主宰著人的精神活動與意識思維，由此可知，古人認為心和大腦活動也有關係。郭大維中醫師強調，趁夏季時將心這個臟腑調理好，受惠不只是心臟及心血管，包括大腦及思緒也變得更清晰、有精神。夏天的末端濕氣會變得特別重，因此中醫將夏末到初秋這段時間從夏季裡劃分出來，稱為「長夏」。雖然夏季養生的重點在養心，但長夏時濕氣較重，所以還必須特別注意脾胃的健康。\n長夏養生首重健脾利濕，因為體內又濕又熱，特別容易導致臟腑上火，出現心火過旺的現象，也會影響心血管循環。因此，長夏時如果能將脾胃調理好，讓體內代謝水分的功能正常，就能清熱降火，進而達到保養心血管的效用。中醫認為「脾為後天之本，為氣血生化之源」

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

<class 'ragas.dataset_schema.EvaluationResult'> {'faithfulness': 1.0000}
<class 'float'> 1.0
  -DECISION: GENERATION IS GROUNDED IN DOCUMENTS-
---GRADE GENERATION vs QUESTION---
  -DECISION: GENERATION ADDRESSES QUESTION-


秋天可以吃梨子、百合、羅漢果、人參和茯苓來潤肺，這些食材有助於補養肺氣與呼吸道，並潤燥。


In [143]:
run("養腎應該吃什麼?")

---ROUTE QUESTION---
  -ROUTETO VECTORSTORE-
---RETRIEVE---
[Document(metadata={'page': None, 'source': None}, page_content='養腎應該吃什麼，「色黑者入腎」，中醫認為冬季養腎要多吃天然黑色食材，像是黑米（紫米）、黑枸杞、栗子、烏骨雞、黑芝麻、黑豆、黑木耳等。郭大維中醫師表示，栗子外形很像腎臟，因此又被稱為「腎之果」，栗子的價格很平民，但滋養身體的效用卻很好，像是脾胃虛弱、腎虛或體虛的長者，都可以多吃。一般較常用來補養腎氣的中藥材，包括了管花肉蓯蓉、何首烏及黃精，因為腎和大腦關係密切，這些藥材除了入腎之外，還可以補腦髓，進而讓記憶力變好。想要顧腦的人，還可以搭配食用銀杏，因為銀杏含有豐富的類黃酮，可以抗氧化、促進血液循環，增加大腦含氧量，因而能增強記憶力。'), Document(metadata={'page': None, 'source': None}, page_content='冬季養腎必學健身操，養生操可以活動肢體，並且調理氣息，達到滋養臟腑的功效。郭大維中醫師建議，利用重訓鍛練核心肌群，可以強化腎氣，或經常做開合跳等動作，能刺激足底的湧泉穴，達到健身強腎的效用。此外，八段錦中的第六式「兩手攀足固腰」，也可以達到固腎精、保養腎陽的效果，因為可以改善腰酸背痛，也很適合久坐、久站的人。'), Document(metadata={'Header 1': '【冬之卷】', 'Header 2': '發行單位', 'Keywords': ['疾病', '發行', '單位']}, page_content='發行單位，Nutrilite紐崔萊執行製作｜良醫健康網※本內容僅提供相關疾病之醫學及衛教資訊；關於個人健康狀況或疾病治療方面的問題，建議向醫護人員諮詢專業意見。'), Document(metadata={'Header 1': '【夏之卷】', 'Header 2': '初夏養心長夏養脾胃', 'Keywords': ['脾胃', '中醫', '臟腑']}, page_content='初夏養心長夏養脾胃，相較於西醫的心臟，中醫的「心」涵蓋的範圍更廣泛。「心主血脈」，表示全身的血脈，也就是所有的血管都歸心所管。不過，《黃帝內經》也指出：「

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

<class 'ragas.dataset_schema.EvaluationResult'> {'faithfulness': 1.0000}
<class 'float'> 1.0
  -DECISION: GENERATION IS GROUNDED IN DOCUMENTS-
---GRADE GENERATION vs QUESTION---
  -DECISION: GENERATION ADDRESSES QUESTION-


養腎應該多吃天然黑色食材，如黑米（紫米）、黑枸杞、栗子、烏骨雞、黑芝麻、黑豆、黑木耳等。栗子被稱為「腎之果」，對於脾胃虛弱、腎虛或體虛的長者特別有益。此外，常用來補養腎氣的中藥材包括管花肉蓯蓉、何首烏及黃精。若想顧腦，也可以搭配食用銀杏，因為銀杏含有豐富的類黃酮，有助於增強記憶力。


In [144]:
run("冬季吃什麼最補腎 下列何者正確:管花肉蓯蓉 西瓜 黑木耳 栗子 銀杏 麻辣鍋")

---ROUTE QUESTION---
  -ROUTETO VECTORSTORE-
---RETRIEVE---
[Document(metadata={'page': None, 'source': None}, page_content='管花肉蓯蓉，又有「沙漠人參」之稱，是珍貴又稀有的中藥材。\n冬季可多食管花肉蓯蓉、銀杏、栗子、黑木耳等食材滋補腎腦。冬季補腎補腎'), Document(metadata={'page': None, 'source': None}, page_content='養腎應該吃什麼，「色黑者入腎」，中醫認為冬季養腎要多吃天然黑色食材，像是黑米（紫米）、黑枸杞、栗子、烏骨雞、黑芝麻、黑豆、黑木耳等。郭大維中醫師表示，栗子外形很像腎臟，因此又被稱為「腎之果」，栗子的價格很平民，但滋養身體的效用卻很好，像是脾胃虛弱、腎虛或體虛的長者，都可以多吃。一般較常用來補養腎氣的中藥材，包括了管花肉蓯蓉、何首烏及黃精，因為腎和大腦關係密切，這些藥材除了入腎之外，還可以補腦髓，進而讓記憶力變好。想要顧腦的人，還可以搭配食用銀杏，因為銀杏含有豐富的類黃酮，可以抗氧化、促進血液循環，增加大腦含氧量，因而能增強記憶力。'), Document(metadata={'Header 1': '【冬之卷】', 'Header 2': '發行單位', 'Keywords': ['疾病', '發行', '單位']}, page_content='發行單位，Nutrilite紐崔萊執行製作｜良醫健康網※本內容僅提供相關疾病之醫學及衛教資訊；關於個人健康狀況或疾病治療方面的問題，建議向醫護人員諮詢專業意見。'), Document(metadata={'Header 1': '【夏之卷】', 'Header 2': '初夏養心長夏養脾胃', 'Keywords': ['脾胃', '中醫', '臟腑']}, page_content='初夏養心長夏養脾胃，相較於西醫的心臟，中醫的「心」涵蓋的範圍更廣泛。「心主血脈」，表示全身的血脈，也就是所有的血管都歸心所管。不過，《黃帝內經》也指出：「心者君主之官也，神明出焉」，意思是心也主宰著人的精神活動與意識思維，由此可知，古人認為心和大腦活動也有關係。郭大維中醫師強調，趁夏季時將心這個臟腑調理好，受惠不只是心臟及心血管，包括

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

<class 'ragas.dataset_schema.EvaluationResult'> {'faithfulness': 1.0000}
<class 'float'> 1.0
  -DECISION: GENERATION IS GROUNDED IN DOCUMENTS-
---GRADE GENERATION vs QUESTION---
  -DECISION: GENERATION ADDRESSES QUESTION-


冬季補腎可以多吃的食材包括管花肉蓯蓉、黑木耳、栗子和銀杏。因此，正確的選擇是管花肉蓯蓉、黑木耳、栗子和銀杏。西瓜和麻辣鍋不在推薦的食材之列。


In [145]:
run("那些飲食習慣傷心又傷脾?")

---ROUTE QUESTION---
  -ROUTETO VECTORSTORE-
---RETRIEVE---
[Document(metadata={'page': None, 'source': None}, page_content='當心！飲食習慣傷心又傷脾，夏季氣候又熱又濕，原本就易耗損心氣及損傷脾胃，如果沒有在飲食及生活上多加留意，除了容易上火之外，也會讓體內濕邪內鬱，對於健康非常不利。郭大維中醫師提醒，夏天應避免下列行為，才能維護心血管及脾胃的健康：一、貪涼吃冰：炎夏酷熱難耐，很多人會大啖冰品、冷飲或屬性較生冷的食物，這些東西吃多了會讓毛孔緊縮，導致人體散熱功能變差，所以會感覺更加燥熱；此外，太過寒涼的食物，會阻礙脾胃的化濕功能，讓濕氣更不易排出體外。二、常吃烤炸辣食物：夏季本來就容易火氣大，若常吃烤炸辣等容易讓臟腑上火的食物，可能會造成「火上加火」的情況。當心火太過旺盛時，血液循環就會加快，汗也\n會流得比較多，心血管的負擔自然也會變重。此外，中醫有「四季脾旺不受邪」的觀念，意思是，人體脾胃功能旺盛，就不易受到病邪的危害。因此，若過度服用烤炸辣等重口味食物，脾胃受損，也會使身體抵抗疾病的能力下降。三、嗜吃高糖或高油類食物：飲食中的油脂及糖分都會增加體內三酸甘油脂的濃度，造成血脂過高的情況，進而增加心血管堵塞及動脈粥狀硬化風險。四、暴飲暴食：飲食不規律，經常暴飲暴食的人，一不小心就會吃進太多食物，若這些熱量沒有被消耗完，就會轉變成三酸甘油脂，提升罹患心血管病變的可能性。五、抽菸喝酒：菸害會造成血管損傷、減少血液中好的膽固醇含量，並且加速動脈硬化的發生；過量的酒精易導致高血壓、心律不整及心肌受損，恐提高罹患心血管疾病的機率。'), Document(metadata={'page': None, 'source': None}, page_content='養脾胃這樣做，中醫有「內傷脾胃，百病由生」說法，人體依賴飲食中的營養，以維持生命的活動，若脾胃的消化、吸收功能失常，即使吃再好的食材，身體也無法從中得到養分。由此可知，只要將脾胃調理好，全身臟腑都能得到滋養，對於提升心臟及心血管的健康也很有助益。養脾胃就要從飲食著手，適時適度、定時定量，是調理脾胃的重要原則。除此之外，郭大維中醫師建議還可以多吃山藥、南瓜及地瓜等黃色或帶天然甘甜味的食材，對於健

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

<class 'ragas.dataset_schema.EvaluationResult'> {'faithfulness': 0.9286}
<class 'float'> 0.9285714285714286
  -DECISION: GENERATION IS GROUNDED IN DOCUMENTS-
---GRADE GENERATION vs QUESTION---
  -DECISION: GENERATION ADDRESSES QUESTION-


以下是一些傷心又傷脾的飲食習慣：

1. **貪涼吃冰**：過多食用冰品、冷飲或生冷食物會讓毛孔緊縮，影響散熱功能，並阻礙脾胃的化濕功能。
2. **常吃烤炸辣食物**：這類食物容易讓臟腑上火，可能造成火氣過旺，增加心血管負擔。
3. **嗜吃高糖或高油類食物**：這些食物會增加體內三酸甘油脂的濃度，增加心血管疾病的風險。
4. **暴飲暴食**：不規律的飲食習慣會導致攝取過多熱量，轉變為三酸甘油脂，提升心血管病變的可能性。
5. **抽菸喝酒**：菸害會損傷血管，酒精過量則可能導致高血壓和心律不整，增加心血管疾病的風險。


In [146]:
# run("養護心血管可以怎麼吃")

In [147]:
# run("養身有句話說""春夏養陽""下句話是?") 

In [148]:
# run("夏季適合保養哪個器官")

In [149]:
# run("腎不健康有哪些徵兆")

In [150]:
# run("腎虛要吃甚麼")

In [151]:
# run("肺不好，如何養肺")